In [1]:
!pip install praw

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 2.6 MB/s eta 0:00:00


In [57]:
!pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 7.6 MB/s eta 0:00:00


In [58]:
import requests
import json
import time
import praw
import datetime
import re
import contractions

In [3]:
import pandas as pd

In [39]:
def fetch_new_posts(reddit, subreddits, limit=5):
  all_posts = {}
  for subreddit_name in subreddits:
    all_posts[subreddit_name] = []
    subreddit = reddit.subreddit(subreddit_name)
    print(f"Fetching posts from r/{subreddit_name}...")
    posts = subreddit.new(limit=limit) # Fetch the latest posts
    count = 0
    for submission in posts:
      if submission is not None and submission.author is not None:
        temp_post_dict = {"id": submission.id, "url":submission.url, "created_utc":submission.created_utc, "author":submission.author.name,  "title":submission.title, "post":submission.selftext, "score":submission.score, "upvote_ratio":submission.upvote_ratio, "num_comments":submission.num_comments}
        count = count+1
        all_posts[subreddit_name].append(temp_post_dict)
    print(count)
    time.sleep(1)
  return all_posts

def save_json(data):
  for key in data.keys():
    with open("/content/{0}.json".format(key), 'w', encoding='utf-8') as json_file:
      json.dump(data[key], json_file, indent=4, ensure_ascii=False)

In [59]:
def load_json(file_path):
  with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)
  return data

def preprocess_text(text):
  text = text.lower() # Convert to lowercase
  text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE) # Remove URLs
  text = re.sub(r'\S*@\S*\s?',' ',text) # Remove emails
  text = re.sub(r'@\S+', ' ', text) # Remove mentions if any
  text = contractions.fix(text) # Expand contractions like don't to do not
  text = re.sub(r'\W', ' ', text) # Allow only word characters
  text = re.sub(r'\s+', ' ', text).strip() # Remove extra spaces
  return text

def prepare_data_for_analysis(json_file, city="default"):
  data = load_json(json_file)
  results = []
  # for posts in data.items():
  for post in data:
    title = preprocess_text(post.get("title", ""))
    content = preprocess_text(post.get("post", ""))
    combined_text = f"{title} {content}"
    results.append({
        "city": city,
        "id": post["id"],
        "url": post["url"],
        "created_utc": post["created_utc"],
        "author": post["author"],
        "text": combined_text,
        "reddit_score": post["score"],
        "reddit_upvote_ratio": post["upvote_ratio"]})
  return results

def save_to_json(data, output_file):
  with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(data, f, indent=4, ensure_ascii=False)

In [49]:
def fetch_posts_via_id(reddit, ids):
  all_posts = {}
  count = 0
  for id in ids:
    submission = reddit.submission(id=id)
    subreddit_name = submission.subreddit.display_name
    if subreddit_name not in all_posts.keys():
      all_posts[subreddit_name] = []
    if submission is not None and submission.author is not None:
      temp_post_dict = {"id": submission.id, "url":submission.url, "created_utc":submission.created_utc, "author":submission.author.name,  "title":submission.title, "post":submission.selftext, "score":submission.score, "upvote_ratio":submission.upvote_ratio, "num_comments":submission.num_comments}
      all_posts[subreddit_name].append(temp_post_dict)
      count = count + 1
      if count%50 == 0: # to limit rate limit
        print(count)
        time.sleep(1)
  return all_posts

In [89]:
def fetch_comments_from_submission(reddit, ids):
  all_comments = {}
  for id in ids:
    submission = reddit.submission(id=id)
    subreddit_name = submission.subreddit.display_name
    if subreddit_name not in all_comments.keys():
      all_comments[subreddit_name] = []
    submission.comments.replace_more(limit=None)
    count = 0
    comment_list = submission.comments.list()
    for comment in comment_list:
      if comment is not None and comment.author is not None:
        temp_comment_dict = {"id": comment.id, "url": comment.permalink, "created_utc": comment.created_utc, "author": str(comment.author.name), "submission_id": comment.link_id, "parent_id": comment.parent_id, "post": comment.body, "score": comment.score}
        all_comments[subreddit_name].append(temp_comment_dict)
        count = count + 1
        if count%50 == 0: # to limit rate limit
          print(count)
          time.sleep(1)
  return all_comments

In [4]:
# Creating a Reddit instance with credentials
reddit = praw.Reddit(
    client_id="kcl92OEqmozGzUpB8JJ5xw",
    client_secret="LPQzJlaUx-hyNtQnWHXLMfyKJYalmg",
    user_agent="social_media_listening_cities_v1.0",
    check_for_async=False)

#subreddits to get the posts from
subreddits = ["bangalore", "mumbai", "delhi", "visakhapatnam", "newyorkcity" ]

file_paths = ["/content/bangalore.json", "/content/delhi.json", "/content/mumbai.json", "/content/newyorkcity.json", "/content/visakhapatnam.json"]
cities = ["bangalore","delhi","mumbai","newyorkcity","visakhapatnam"]


In [ ]:
# Fetch new posts from subreddits

# all_posts = fetch_new_posts(reddit=reddit, subreddits=subreddits, limit=2000)

# Save posts into json files
# save_json(all_posts)

In [54]:
# Use reddit post ids to fetch data and save into json

# file_paths = ["/content/raw_bangalore.json", "/content/raw_delhi.json", "/content/raw_mumbai.json", "/content/raw_newyorkcity.json", "/content/raw_visakhapatnam.json"]

# for file in file_paths:
#   df = pd.read_json(file)
#   ids = df['id'].values.tolist()
#   all_posts = fetch_posts_via_id(reddit=reddit, ids=ids)
#   save_json(all_posts)
#   time.sleep(20)

50
100
150
200
250
300
350
400
450
500
550
600
650


In [60]:
# Initial Preprocess reddit data

for i in range(len(file_paths)):
  output_file = f"/content/processed_{cities[i]}.json"
  processed_data = prepare_data_for_analysis(file_paths[i], city=cities[i])
  save_to_json(processed_data, output_file)

In [92]:
# load reddit comments using post ids

for file in file_paths:
  df = pd.read_json(file)
  ids = df['id'].values.tolist()
  all_comments = fetch_comments_from_submission(reddit=reddit, ids=ids)
  save_json(all_comments)
  print(f"{file} completed")
  time.sleep(20)

50
50
50
50
100
150
50
50
50
50
50
50
50
/content/visakhapatnam.json completed
